### Importación de las librerías

In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Configuración inicial

In [ ]:
meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "es-ES,es;q=0.9",
    "Referer": "https://www.lajornadamaya.mx/",
}
directorio = "" #Direcctorio donde se guardará el CSV
# Crear un DataFrame vacío para almacenar todas las noticias
df_noticias = pd.DataFrame(columns=["Título", "Fecha", "Contenido"])

### Función para obtener los enlaces de las noticias por mes

In [12]:
def get_news_links(mes, headers):
    # URL base para la letra
    base_url = "https://www.lajornadamaya.mx/k'iintsil/"
    href_list = set()  # Conjunto para evitar duplicados

    url = f"{base_url}{mes}-2019"
    print(f"Obteniendo página: {url}")  # Verificación de la URL actual
    response = requests.get(url, headers=headers)

    # Verificar si la respuesta fue exitosa
    if response.status_code != 200:
        print(f"Error al acceder a la página: {response.status_code}")
        return list(href_list)

    content = response.content
    soup = BeautifulSoup(content, 'html.parser')

    # Buscar todas las etiquetas <a> dentro de <h5>
    for h5 in soup.find_all('h5'):
        a = h5.find('a', href=True)
        if a:
            link = a['href']
            if link.startswith('http'):
                href_list.add(link)  # Usa add() para asegurarte de no agregar duplicados

    return list(href_list)  # Convertir el conjunto en lista para el resultado

### Función para obtener los datos de cada noticia

In [13]:
def getInfo(link, headers):
    # Hacer la solicitud HTTP
    response = requests.get(link, headers=headers)

    if response.status_code != 200:
        print(f"Error al acceder a {link}: {response.status_code}")
        return None  # Si falla, regresamos None

    # Analizar el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extraer título
    titulo_tag = soup.find('a', class_='post-headline')
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else "Sin título"

    # Extraer la fecha correctamente
    fecha = "Sin fecha"
    fecha_tag = soup.find('p', class_='maya')
    if fecha_tag:
        fecha_siguiente = fecha_tag.find_next_sibling('p')  # Tomamos el siguiente <p> con la fecha
        if fecha_siguiente:
            fecha = fecha_siguiente.get_text(strip=True).split('|')[0].strip()

    # Extraer el contenido después de la fecha
    contenido = []
    contenido_iniciado = False

    for p in soup.find_all('p'):
        texto_parrafo = p.get_text(separator=' ', strip=True)

        # Saltamos la fecha y los párrafos previos
        if fecha_siguiente and p == fecha_siguiente:
            contenido_iniciado = True
            continue

        # Si ya pasamos la fecha, agregamos los párrafos al contenido
        if contenido_iniciado and texto_parrafo:
            contenido.append(texto_parrafo)

    return titulo, fecha, contenido

### Extracción de la información del año 2019

In [14]:
#Esta sería la compilación completa
# Iterar sobre cada mes
for mes in meses:
    links_mes = sorted(get_news_links(mes, headers), reverse=True)# Obtener links del mes ordenados
    
    for link in links_mes:
        noticia = getInfo(link, headers)  # Extraer la info de la noticia
        
        if noticia:
            df_noticias.loc[len(df_noticias)] = noticia  # Agregar al DataFrame

Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/enero-2019
Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/febrero-2019
Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/marzo-2019
Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/abril-2019
Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/mayo-2019
Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/junio-2019
Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/julio-2019
Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/agosto-2019
Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/septiembre-2019
Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/octubre-2019
Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/noviembre-2019
Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/diciembre-2019


### Guardado del CSV

In [15]:
df_noticias.to_csv(directorio + 'noticias.csv', index=False)


# *Pruebas

In [16]:
df_noticias[['Contenido']]

,Contenido
0,[Fabiola Palapa Quijas Oochel: Editorial Tauru...
1,[AFP Oochel Rodrigo Díaz Guzmán K'iintsill Dav...
2,[Carlos Paul Oochel Cortesía de la Fonoteca/ C...
3,[Blanche Petrich Oochelo'ob Jesús Villaseca / ...
4,[Oochel Cortesía del Museo K'iintsil Lunes 28 ...
...,...
63,[Mónica Mateos-Vega y Juan Carlos G. Partida O...
64,[Reuters OOchel Time K'iintsil Jueves 12 ti' d...
65,[Notimex OOchel Mauricio Murat K'iintsil Viern...
66,[Fabiola Martínez Oochel Fernando Eloy K'iints...


In [17]:
#Esto es una prueba
links_mes = sorted(get_news_links('noviembre', headers), reverse=True) # Obtener links del mes y ordenarlos
links_mes

Obteniendo página: https://www.lajornadamaya.mx/k'iintsil/noviembre-2019


["https://www.lajornadamaya.mx/k'iintsil/114418/k-ex-ma-liik-ik-kaaj-tuka-ateene-ku-beeta-al-xandu-u-k-iimbesajil-pixano-ob-tu-kaajil-juchitan",
 "https://www.lajornadamaya.mx/k'iintsil/114282/k-aak-e-tu-chiimpoltaj-u-najil-k-uj-u-ko-olebil-guadalupe-ich-notre-dame",
 "https://www.lajornadamaya.mx/k'iintsil/114186/jts-iib-k-ajla-ayil-its-ato-obe-ku-ki-imak-ooltiko-ob-oochel-tukulta-an-ku-nup-ik-da-vinci-yeetel-maquiavelo",
 "https://www.lajornadamaya.mx/k'iintsil/114114/ya-abchaj-u-goorilailo-ob-wiits-ba-ale-laayli-ich-sajbe-entsil-yaniko-ob",
 "https://www.lajornadamaya.mx/k'iintsil/114043/kaxta-ab-tu-kaajil-tultepece-u-yaax-traampasil-mamut-yanchaj-yook-ol-kaab",
 "https://www.lajornadamaya.mx/k'iintsil/114024/u-tsikbalil-tsiimin-k-aak-",
 "https://www.lajornadamaya.mx/k'iintsil/113779/cirque-du-soleile-ku-chiimpoltik-leo-messi-yeetel-cha-an-yook-lal-u-kuxtal",
 "https://www.lajornadamaya.mx/k'iintsil/113680/yook-lal-u-muuk-il-ulaak-muuch-kabilo-obe-taan-u-xu-ulul-yaax-maseewal-jala-

In [18]:
#Esto es una prueba
noticia = getInfo(links_mes[2], headers) #Obtener la información de la noticia
noticia

('Jts’íib k’ajla’ayil its’ato’obe’ ku ki’imak óoltiko’ob oochel tukulta’an ku nup’ik Da Vinci yéetel Maquiavelo',
 '05/11/2019',
 ["AFp Oochel K'iintsil Martes 5 ti' noviembre, 2019 Rennes. U yoochel ti’ u táan u yich juntúul k’olis máak yaan u me’exe’ tu ts’áaj tuukul ichil ajts’íib k’ajla’ayo’ob yéetel máaxo’ob uts tu yicho’ob its’at, tumen ku tukultiko’obe’ tu kaxto’ob jump’éel k’a’anan meyaj mix juntéen yáax ila’ak ichil u máan ja’ab’ob. Le je’ela’ ti’ yaan ka’ach ichil junmúuch’ meyajo’ob yano’ob ichil jump’éel káastiyo yaan tu k’íiwikil Francia. Jump’éel k’uben t’aan ts’íibta’an 145 ja’abo’ob paachil yéetel jts’a’ab k’ajóoltbil tu ja’abil 2018e’, ku ya’alik ma’ xaan wa le ku chikpajalo’ u yoochel Nicolás Maquiavelo, juntúul filoosofóo italianoil kuxlaj ichil u siigloil XV yéetel XVI, leti’ u yuumil El príncipe, ba’ale’ boonile’ beeta’ab tumen réenasentistaa jka’ansaj Leonardo da Vinci. Kex tumen táan u binetik ya’abach winalo’ob ti’al u beeta’al xaak’alilo’ob ti’al u yila’al máax